# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


---
# Projet A2023 : Prédiction du nombre de passages sur le REV 


L’analyse des données ouvertes de la circulation sur la rue Saint-Denis révèle une hausse importante de l’utilisation du vélo par rapport aux autres véhicules, au tournant de l’installation du Réseau express vélo (REV) en 2020 comme le rapporte le journal *Le Devoir* dans cet [article](https://www.ledevoir.com/environnement/735849/le-ratio-velo-auto-s-ameliore-sur-le-rev-saint-denis). Serez-vous en mesure de prédire l'achalandage du REV pour l'année 2023, de janvier à septembre ? Est-ce que les conditions météorologiques ont une influence sur le nombre de passages ? Ce sont des questions que vous aurez l'occasion d'étudier dans le cadre de ce projet.

**But** : Prédire le nombre de passages journalier sur le REV en direction nord à l'intersection de la rue St-Denis et de la Piste des Carrières pour tous les jours entre le 1er janvier 2023 et le 30 septembre 2023.

**Objectifs spécifiques non exhaustifs**
1. Effectuer une analyse exploratoire des données afin d'extraire qualitativement certains les potentiels liens entre les variables.
2. Élaborer plusieurs modèles prédictifs pour le nombre de passages journaliers sur le REV en fonction des différentes variables explicatives à disposition (nombre de passages sur d'autres tronçons, conditions météorologiques, etc.).
3. Sélectionner le meilleur modèle prédicitif à l'aide d'un critère choisi.
4. Exploiter le modèle choisi pour effectuer vos prédictions.

Ces étapes peuvent être recommencées autant de fois que nécessaire afin d'obtenir le meilleur modèle prédictif possible.

La description du projet est disponible à l'adresse suivante : https://www.kaggle.com/t/b2f3163a94434f0da9e532139744c49d

Ce calepin Jupyter de base permet de charger les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

### Données

Dans un premier temps, vous devrez récupérer les données sur Kaggle. Les fichiers disponibles sont les suivants :
- train.csv
- test.csv

**Déposez ces fichiers dans le répertoire de ce calepin.**

Le fichier *train.csv* contient les conditions météorologiques ainsi que le nombre de passages enregistrés sur le tronçon du REV à l'intersection St-Denis/Castelnau (45.53905N,-73.61687W). Voici la description des variables :

- Date : date
- MaxTemp : Température maximale (°C)
- MinTemp : Température minimale (°C)
- MeanTemp : Température moyenne (°C)
- TotalRain : Accumulation totale de pluie (mm)
- TotalSnow : Accumulation totale de neige (cm)
- TotalPrecip : Accumulation totale de précipitations (pluie + neige, mm)
- SnowGrnd : Couvert de neige au sol (cm)
- SpdGust : Vitesse maximale des rafales de vent (km/h)
- REV : Nombre de passages de vélo au compteur sur le REV en direction sud à l'intersection St-Denis/Castelnau 

Le fichier *test.csv* contient les conditions météorologiques pour l'année 2023 mais pas le nombre de passages sur le REV, la variable que vous devrez prédire. Le fichier contient aussi l'identifiant (:ID) pour les prédictions. La qualité de vos prédictions sera ensuite évaluée avec le [*Root Mean Squared Log Error*](https://www.kaggle.com/code/carlolepelaars/understanding-the-metric-rmsle) lorsque vous les téléverserez sur Kaggle. Vos prédictions seront comparées à celles des autres équipes de la classe.


### Consignes

- Vous devez constituer une équipe de 3 à 5 personnes.
- Au moins une solution doit être proposée sur Kaggle.
- Utilisez votre identifiant d'équipe pour téléverser vos prédictions sur Kaggle.
- Un seul calepin *.ipynb* par équipe doit être remis. Ce fichier devra documenter et illustrer la procédure qui vous a permis de produire vos meilleures prédictions. Ce fichier constitue le rapport final du projet.
- Le langage Julia doit être utilisé.
- Votre démarche doit être rigoureusement justifiée (consultez la grille de correction pour vous orienter).

### Quelques conseils

Votre calepin doit permettre à une personne à l'extérieur de l'équipe de comprendre votre démarche et de reproduire vos résultats. Par exemple, une bonne façon de faire consiste à expliquer dans une cellule de texte la tâche qui est accomplie dans la cellule de code suivante. 

Je vous encourage fortement à faire une analyse exploratoire des données pour développer une meilleure expertise sur le problème. C'est une étape qui est toujours négligée mais qui est essentielle. C'est avec l'analyse exploratoire que vous viendra des idées d'amélioration, comme par exemple créer de nouvelles variables explicatives.

Vous pouvez utiliser directement tout ce qui se trouve dans les notes de cours sans explication et toutes les librairies utilisées dans le cours (incluant mes fonctions).

Ce calepin contient un modèle très simple de prédiction : on n'utilise qu'une seule variable explicative. Ce sera votre travail d'améliorer ces prédictions avec la méthode et les variables de votre choix.

S'il y a des données manquantes, ce sera à vous de traiter ce problème. Vous devriez développer une méthode d'imputation (de remplacement) des données manquantes.

Attention aux données aberrantes. Elles peuvent faire dérailler tous le modèle prédictif si elle ne sont pas prises en compte.

Prenez la peine de documenter succinctement les essais infructueux. Ce n'est pas nécessaire de les expliquer en détails, mais c'est important de les mentionner dans la discussion avec une raison possible de leur échec. De cette façon, une personne qui reprendra votre travail dans le futur ne perdra pas de temps à réessayer une méthode infructueuse déjà testée.

Vous pouvez aussi indiquer dans votre rapport les raisons qui vous font croire pourquoi une méthode a moins bien performée de ce qui était attendu. Vous pouvez également mentionner ce que vous auriez pu tenter si vous aviez eu plus de temps ou plus de données. L'idée est de guider l'analyste qui prendrait la relève de votre travail.

Vous êtes limités à deux soumissionspar jour et par équipe sur Kaggle. Je vous suggère donc de bien tester vos modèles localement et de ne téléverser que vos meilleurs prédictions de la journée.

In [ ]:
using CSV, DataFrames, Dates, Gadfly, GLM, Statistics

---
## 1. Chargement de données

In [ ]:
train = CSV.read("train.csv", DataFrame)
first(train, 5)

---
## 2. Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de se familiariser avec les données. C'est une analyse exploratoire sommaire. Je vous encourage fortement à poursuivre cette analyse.

#### 2.1 Nombre de passages sur le REV en fonction de la date

In [ ]:
set_default_plot_size(12cm, 10cm)
plot(train, x=:Date, y=:REV)

#### 2.2 Nombre de passages sur le REV en fonction des conditions météorologiques

In [ ]:
set_default_plot_size(16cm, 12cm)

fig1 = plot(train, x=:MeanTemp, y=:REV)
fig2 = plot(train, x=:TotalPrecip, y=:REV)
fig3 = plot(train, x=:SnowGrnd, y=:REV)
fig4 = plot(train, x=:SpdGust, y=:REV)

gridstack([fig1 fig2; fig3 fig4])

---
## 3. Ajustement d'un modèle de régression linéaire

Pour cet exemple simple, on n'utilise que la température moyenne pour prédire le logarithme des passages sur le REV avec la régression linéaire. On pourrait aussi pu utiliser un modèle linéaire généralisé pour modéliser le nombre de passages avec la loi de Poisson.

#### 3.1. Transformation du nombres de passages à l'échelle logarithmique

In [ ]:
data = deepcopy(train)
data[!,:REV] = log.(train.REV)
first(data,5)

#### 3.2. Ajustement du modèle de régression linéaire

In [ ]:
model = lm(@formula(REV ~ MeanTemp), data)

#### 3.3. Validation graphique

Pour avoir un aperçu de la qualité de la regression, on ajoute la droite de régression.

In [ ]:
β̂ = coef(model)

points = layer(data, x=:MeanTemp, y=:REV, Geom.point)
line = layer(x->β̂[1]+β̂[2]*x, -25, 35, Theme(default_color="red"))

plot(line, points)

---
## 4. Estimation du nombre de passages de l'ensemble de test

On utilise le modèle simple de la section précédente pour le nombre de passages pour chacun des jours de l'ensemble de test.

#### 4.1 Chargement des données de l'ensemble de test

In [ ]:
test = CSV.read("test.csv", DataFrame)
first(test, 5)

#### 4.2 Traitement des valeurs manquantes

Dans l'ensemble de test, il y a des jours pour lesquelles la température moyenne (la variable explicative utilisée dans le modèle simple) est manquante. 

Ici, je propose de remplacer les valeurs manquantes par la température moyenne du jour précédent. C'est une façon simple et même peut-être simpliste de traiter les valeurs manquantes. Ce sera à vous de décider comment les traiter.

In [ ]:
# Nombre de jours où il manque la température moyenne
count(ismissing.(test.MeanTemp))

In [ ]:
"""
    coalesceWithPrevious!(x::AbstractArray{T} where T <: Union{Missing, Real})
    
Coalesce missing values with the previous non-missing.

### Details

If the first element of `x` is missing, than it is replace with the first non-missing value.
"""
function coalesceWithPrevious!(x::AbstractArray{T} where T <: Union{Missing, Real})
    
    if ismissing(x[1])
        x[1] = x[findfirst(.!(ismissing.(x)))]
    end
        
    ind = findall(ismissing.(x))
    
    for i in ind
        x[i] = x[i-1]
    end
    
    return x
    
end

In [ ]:
# Remplacement des valeurs manquantes par la valeur précédente.
coalesceWithPrevious!(test.MeanTemp)

# Nombre de valeurs manquantes après le remplacement
count(ismissing.(test.MeanTemp))

#### 4.3 Prédictions

On retransforme les prédictions dans l'espace logarithmique pour obtenir des nombres de passages.

In [ ]:
predictions = exp.(predict(model, test))

# Vous pouvez transformer vos prédictions en valeurs entières si vous le souhaitez. Mais ce n'est pas nécessaire.
# predictions = Int.(round.(predictions, digits=0))

#### 4.4 Préparation du fichier des préditions pour téléverser sur Kaggle

Le fichier *benchmark_predictions.csv* généré peut être téléversé sur Kaggle. Il est composé d'une colonne d'identifiants (:ID) et d'une colonne de la prédiction (:REV).

In [ ]:
benchmark_predictions = DataFrame(ID = test.ID, REV=predictions)
first(benchmark_predictions, 5)

In [ ]:
CSV.write("benchmark_predictions.csv", benchmark_predictions)